In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
import torch
torch.set_float32_matmul_precision('medium')

In [3]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40
from datasets.shapenet.shapenet_pointflow_sparse import get_dataloaders

path = "../data/ModelNet40"
path = "../data/ShapeNetCore/pointclouds"
categories = ['airplane']

tr, te = get_dataloaders(path, categories=categories)

(1, 1, 1)
Total number of data:2832
Min number of points: (train)2048 (test)2048
(1, 1, 1)
Total number of data:405
Min number of points: (train)2048 (test)2048


/opt/conda/envs/spvd/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [4]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features": [32, 64, 128, 192, 192, 256],
    "num_layers": 1,
    "attn_heads": (None, None, None, 8, 8)
}]
up_blocks = [
    {
        "features": [256, 192, 192],
        "num_layers": 1,
        "attn_heads": 8,
    },
    {
        "features": [192, 128, 64, 32],
        "num_layers": 1,
        "attn_heads": None
    },
]
t_emb_features = 64

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

import models

model = models.DiffusionBase(model, lr=1e-4)

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [5]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_name = "-".join(categories) if categories else "all"
checkpoint_callback = ModelCheckpoint(
    dirpath=f'../checkpoints/ModelNet40/{checkpoint_name}',
    filename="{epoch:02d}-{val_loss:.2f}", 
    every_n_epochs=5
)

epochs = 50
trainer = L.Trainer(
    log_every_n_steps=5,
    max_epochs=epochs, 
    gradient_clip_val=10.0, 
    callbacks=[checkpoint_callback]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [16]:
from models import SPVD

model = SPVD()
model = models.DiffusionBase(model, lr=1e-4)

In [6]:
trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=te)

/opt/conda/envs/spvd/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ubuntu/SPVD_Lightning/checkpoints/ModelNet40/airplane exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 35.0 M | train
------------------------------------------
35.0 M    Trainable params
0         Non-trainable params
35.0 M    Total params
139.937   Total estimated model params size (MB)
372       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/opt/conda/envs/spvd/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=50` reached.


In [10]:
import torch

torch.save(model.state_dict(), "../checkpoints/ModelNet40/airplane-cross.ckpt")

In [12]:
model = model.cuda().eval()

from utils.schedulers import DDPMSparseSchedulerGPU
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler

ddpm_sched = DDPMSparseScheduler(beta_min=0.0001, beta_max=0.02)

In [13]:
from utils.visualization import visualize_notebook

preds = ddpm_sched.sample(model, 16, 2048)

Sampling: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [01:07<00:00, 15.11it/s]


AttributeError: 'SparseTensor' object has no attribute 'reshape'

In [9]:
visualize_notebook(preds, x_offset=7, y_offset=7, point_size=0.2)

Output()